# Minimize the absolute value
A stage-wise independent finite discrete MSLP.    
The data process is stage-wise independent and on the RHS.    
This is a problem originally from Bernado.        

## Formulation
\begin{align*}
    \min~ & \sum_{t=1}^T |x_t|\\
    s.t.~ & \textrm{for}~ t = 1,\dots,T\\
         & x_t = x_{t-1} + c_t + \epsilon_t\\
         & \epsilon_t\overset{i.i.d}{\sim} N(-1,1) \\
         & c_t \in \{-1,1\} \\
         & x_1 = 0
\end{align*}

In [1]:
from msppy.msp import MSIP
from msppy.solver import Extensive, SDDiP
import numpy
numpy.random.seed(3)
T = 4
precision = 1
rhs = [numpy.random.normal(-1,1,size=10) for _ in range(T-1)]
absolute = MSIP(T=T, bound=0)
for t in range(T):
    m = absolute[t]
    x_now, x_past = m.addStateVar(lb=-100, ub=100, name='x')
    if t == 0:
        m.addConstr(x_now == 0)
    else:
        y = m.addVar(name='y', obj=1)
        c = m.addVar(vtype='B', name='c')
        slack = m.addVar(ub=1/10**(precision), name='slack')
        m.addConstr(y >= -x_now)
        m.addConstr(y >= x_now)
        m.addConstr(
            x_now - x_past - 2*c  + slack == -1,
            uncertainty={'rhs': rhs[t-1]}
        )

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only


In [2]:
absolute.binarize(bin_stage=T, precision=precision)
Extensive(absolute).solve(outputFlag=0)

Academic license - for non-commercial use only


1.8647999999999987

In [3]:
absolute_sddip = SDDiP(absolute)
absolute_sddip.solve(cuts=['LG'], n_steps=3, n_processes=3, max_iterations=30)

------------------------------------------------------------------------------------
                             SDDP Solver, Lingquan Ding                             
------------------------------------------------------------------------------------
   Iteration               Bound            Value 95% CI (3)                    Time
------------------------------------------------------------------------------------
           1            1.009997           0.061104, 3.538896               5.660007
           2            1.088501           0.603172, 1.596828               5.847554
           3            1.330106           1.694266, 3.439067               5.715516
           4            1.560988           0.999104, 3.067562               7.596119
           5            1.688548           0.958573, 3.508094               7.895199
           6            1.710439           0.399116, 3.267551               7.554958
           7            1.771223           1.208172, 2.458494    